In [1]:
import pandas as pd

csv_url = "https://health-infobase.canada.ca/src/data/covidLive/covid19.csv"
df = pd.read_csv(csv_url)


In [2]:
#parse government data so it is usable
df['dates'] = pd.to_datetime(df['date'], dayfirst = True)
df['YMD'] = df['dates'].dt.date
df = df.drop(['date', 'dates', 'prnameFR', 'pruid'], axis = 1)
#if provinces don't report active cases, assume they have 0, so the data will plot
df['numactive'].fillna(0.0, inplace = True)

#List of unique provinces, Canada, ordered alphabetically with Canada first
provnames = df.prname.unique()
provnames = list(provnames)
provnames.pop()
provnames.sort()
myorder = [2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
provnames = [provnames[i] for i in myorder]
dropnames = ['Canada', 'Nunavut', 'Yukon', 'Northwest Territories']
select_provs = [x for x in provnames if x not in dropnames]

#function to make province-wise dataset
def make_dataset(pnames):
    
    by_prov = pd.DataFrame()

    # Iterate through all the provinces
    for i in pnames:

        # Subset to the province
        sub = df[df['prname'] == i]

        # Add to the overall dataframe
        by_prov = by_prov.append(sub)
    
    return by_prov

#must pivot datasets for linear plot data so prov names are columns
def format_dataset(df, value):
    df_pivot = pd.pivot_table(df,
    columns = 'prname',
    index = 'YMD',
    values = value)
    
    df_pivot = df_pivot.rename_axis(None)

    return df_pivot

In [3]:
dataset = format_dataset(make_dataset(select_provs), 'numtoday')
index_list = pd.date_range(start = '2021-03-26', end = '2021-04-23', closed = 'right')
month_period = dataset.loc[index_list, :]
#print(month_period)
month_total = month_period.sum(axis = 0)
print(month_total)
#month_total.to_csv('data/29JAN_19FEB_totals.csv', encoding = 'utf8')

prname      Alberta  British Columbia  Manitoba  New Brunswick  \
2021-03-27    668.0             936.0      57.0           12.0   
2021-03-28    644.0             805.0      55.0            6.0   
2021-03-29    545.0             777.0      53.0           10.0   
2021-03-30    576.0             840.0      76.0           14.0   
2021-03-31    871.0            1013.0      64.0           12.0   
2021-04-01    875.0             832.0      56.0           10.0   
2021-04-02   1081.0            1074.0      91.0            9.0   
2021-04-03   1071.0            1077.0      90.0            9.0   
2021-04-04    948.0             999.0      67.0           11.0   
2021-04-05    887.0             890.0      68.0           10.0   
2021-04-06    931.0            1068.0      60.0            3.0   
2021-04-07   1351.0             997.0     107.0           14.0   
2021-04-08   1429.0            1293.0     136.0            7.0   
2021-04-09   1521.0            1262.0     179.0            8.0   
2021-04-10